## Phase 1: Set-up

In [1]:
#  !pip install pandas-profiling
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.1 MB/s eta 0:00:00


In [2]:
#module imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import random

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler,LabelEncoder, RobustScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split,cross_val_score, learning_curve

# model imports
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.utils.class_weight import compute_class_weight


# processing imports
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score, precision_score, recall_score


In [3]:
train_data=pd.read_csv('traindata.csv')
train_data.head()

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level,Class
0,0,0,tcp,http,SF,295,2584,0,0,0,...,1.00,0.00,0.01,0.03,0.0,0.0,0.00,0.01,21,0
1,1,0,tcp,http,SF,312,839,0,0,0,...,1.00,0.00,0.00,0.02,0.0,0.0,0.01,0.01,21,0
2,2,0,tcp,ftp,SF,1225,2451,0,0,0,...,0.45,0.02,0.00,0.00,0.0,0.0,0.00,0.00,11,1
3,3,0,tcp,http,SF,293,296,0,0,0,...,1.00,0.00,0.00,0.02,0.0,0.0,0.00,0.02,21,0
4,4,0,tcp,X11,REJ,0,0,0,0,0,...,0.01,0.03,0.01,0.00,0.0,0.0,0.01,1.00,11,0


In [4]:
test_data=pd.read_csv('testdata.csv')
test_data.head()

,ID,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,level
0,0,4,tcp,pop_3,SF,30,93,0,0,0,...,232,0.91,0.04,0.00,0.00,0.0,0.0,0.0,0.0,16
1,1,0,tcp,private,S0,0,0,0,0,0,...,5,0.02,0.08,0.00,0.00,1.0,1.0,0.0,0.0,21
2,2,0,udp,private,SF,52,54,0,0,0,...,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,17
3,3,0,tcp,http,SF,367,588,0,0,0,...,255,1.00,0.00,0.12,0.06,0.0,0.0,0.0,0.0,21
4,4,0,icmp,ecr_i,SF,1008,0,0,0,0,...,255,1.00,0.00,1.00,0.00,0.0,0.0,0.0,0.0,18


#EDA

In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23002 entries, 0 to 23001
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           23002 non-null  int64  
 1   duration                     23002 non-null  int64  
 2   protocol_type                23002 non-null  object 
 3   service                      23002 non-null  object 
 4   flag                         23002 non-null  object 
 5   src_bytes                    23002 non-null  int64  
 6   dst_bytes                    23002 non-null  int64  
 7   land                         23002 non-null  int64  
 8   wrong_fragment               23002 non-null  int64  
 9   urgent                       23002 non-null  int64  
 10  hot                          23002 non-null  int64  
 11  num_failed_logins            23002 non-null  int64  
 12  logged_in                    23002 non-null  int64  
 13  num_compromised 

In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22501 entries, 0 to 22500
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           22501 non-null  int64  
 1   duration                     22501 non-null  int64  
 2   protocol_type                22501 non-null  object 
 3   service                      22501 non-null  object 
 4   flag                         22501 non-null  object 
 5   src_bytes                    22501 non-null  int64  
 6   dst_bytes                    22501 non-null  int64  
 7   land                         22501 non-null  int64  
 8   wrong_fragment               22501 non-null  int64  
 9   urgent                       22501 non-null  int64  
 10  hot                          22501 non-null  int64  
 11  num_failed_logins            22501 non-null  int64  
 12  logged_in                    22501 non-null  int64  
 13  num_compromised 

In [7]:
train_data.nunique()

ID                             23002
duration                         702
protocol_type                      3
service                           66
flag                              11
src_bytes                       1578
dst_bytes                       3657
land                               2
wrong_fragment                     3
urgent                             2
hot                               20
num_failed_logins                  5
logged_in                          2
num_compromised                   26
root_shell                         2
su_attempted                       3
num_root                          25
num_file_creations                20
num_shells                         2
num_access_files                   7
num_outbound_cmds                  1
is_host_login                      1
is_guest_login                     2
count                            461
srv_count                        408
serror_rate                       70
srv_serror_rate                   52
r

In [8]:
test_data.nunique()

ID                             22501
duration                         624
protocol_type                      3
service                           64
flag                              11
src_bytes                       1145
dst_bytes                       3645
land                               2
wrong_fragment                     3
urgent                             4
hot                               16
num_failed_logins                  5
logged_in                          2
num_compromised                   23
root_shell                         2
su_attempted                       3
num_root                          20
num_file_creations                 9
num_shells                         4
num_access_files                   5
num_outbound_cmds                  1
is_host_login                      2
is_guest_login                     2
count                            494
srv_count                        457
serror_rate                       88
srv_serror_rate                   82
r

##Phase 2:Data pre-processing

### Checking for Null values

In [9]:
# Check for duplicate rows across all columns
print("Number of NULL values:\n", train_data.isnull().sum())

Number of NULL values:
 ID                             0
duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate      

In [10]:
# Check for duplicate rows across all columns
print("Number of NULL values:\n", test_data.isnull().sum())

Number of NULL values:
 ID                             0
duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
rerror_rate                    0
srv_rerror_rate                0
same_srv_rate      

### Checking for Duplicate Rows

In [11]:
# Check for duplicate rows across all columns
print("Number of Duplicate Rows:", train_data.duplicated().sum())

Number of Duplicate Rows: 0


In [12]:
# Check for duplicate rows across all columns
print("Number of Duplicate Rows:", test_data.duplicated().sum())

Number of Duplicate Rows: 0


###Drop unnecessary features

In [13]:
columns_to_drop = ["ID","duration"]
train_data.drop(columns=columns_to_drop, axis=1, inplace=True)

In [14]:
test_data.drop(columns=columns_to_drop, axis=1, inplace=True)
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22501 entries, 0 to 22500
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   protocol_type                22501 non-null  object 
 1   service                      22501 non-null  object 
 2   flag                         22501 non-null  object 
 3   src_bytes                    22501 non-null  int64  
 4   dst_bytes                    22501 non-null  int64  
 5   land                         22501 non-null  int64  
 6   wrong_fragment               22501 non-null  int64  
 7   urgent                       22501 non-null  int64  
 8   hot                          22501 non-null  int64  
 9   num_failed_logins            22501 non-null  int64  
 10  logged_in                    22501 non-null  int64  
 11  num_compromised              22501 non-null  int64  
 12  root_shell                   22501 non-null  int64  
 13  su_attempted    

##Phase 3: Data Modeling and Prediction

###Spliting  Data

In [15]:
X_train = train_data.drop('Class', axis=1)
y_train = train_data['Class']

X_test = test_data

###Encoding Categorical Featuers

In [16]:
# Define the columns to be one-hot encoded
columns_to_encode = ['protocol_type', 'service', 'flag']

# Initialize the OneHotEncoder with handle_unknown='ignore' to handle unknown categories
encoder = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')

# Fit and transform the encoder on the training data
X_train_encoded = encoder.fit_transform(X_train[columns_to_encode])
X_train = X_train.drop(columns_to_encode, axis=1)
X_train = pd.concat([X_train, pd.DataFrame(X_train_encoded)], axis=1)

# Transform the encoder on the test data
X_test_encoded = encoder.transform(X_test[columns_to_encode])
X_test = X_test.drop(columns_to_encode, axis=1)
X_test = pd.concat([X_test, pd.DataFrame(X_test_encoded)], axis=1)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


##Phase 3: Data Modeling and Prediction

###Hyper Parameters

In [17]:
# # Define the hyperparameter grid for CatBoost
# catboost_param_grid = {
#     # 'iterations': [10, 20, 30, 40, 50],  # Number of iterations
#     'iterations': [70,90],  # Number of iterations
#     'depth': [4, 6, 8],                 # Depth of the trees
#     'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
# }

# # # Define the hyperparameter grid for AdaBoost
# # ada_param_grid = {
# #     'n_estimators': [50, 100, 200],  # Number of estimators (trees)
# #     'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
# # }

# # # Define the hyperparameter grid for XGBoost
# # xgb_param_grid = {
# #     'n_estimators': [50, 100, 200],  # Number of estimators (trees)
# #     'max_depth': [4, 6, 8],          # Maximum tree depth
# #     'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
# # }

# # # Define the hyperparameter grid for Gradient Boosting
# # gb_param_grid = {
# #     'n_estimators': [50, 100, 200],      # Number of boosting stages (trees)
# #     'learning_rate': [0.01, 0.1, 0.2],  # Learning rate
# #     'max_depth': [3, 4, 5],             # Maximum depth of the individual trees
# # }

# # # Create CatBoost, AdaBoost, XGBoost, and Gradient Boosting classifiers
# catboost_classifier = CatBoostClassifier(random_state=42)
# # ada_boost_classifier = AdaBoostClassifier(random_state=42)
# # xg_boost_classifier = XGBClassifier(use_label_encoder=False, random_state=42)
# # gb_classifier = GradientBoostingClassifier(random_state=42)

# # Initialize GridSearchCV with CatBoost and the hyperparameter grid
# catboost_grid_search = GridSearchCV(estimator=catboost_classifier, param_grid=catboost_param_grid, scoring='accuracy', cv=5)
# catboost_grid_search.fit(X_train, y_train)

# # # Initialize GridSearchCV with AdaBoost and the hyperparameter grid
# # ada_grid_search = GridSearchCV(estimator=ada_boost_classifier, param_grid=ada_param_grid, scoring='accuracy', cv=5)
# # ada_grid_search.fit(X_train, y_train)

# # # Initialize GridSearchCV with XGBoost and the hyperparameter grid
# # xgb_grid_search = GridSearchCV(estimator=xg_boost_classifier, param_grid=xgb_param_grid, scoring='accuracy', cv=5)
# # xgb_grid_search.fit(X_train, y_train)

# # # Initialize GridSearchCV with Gradient Boosting and the hyperparameter grid
# # gb_grid_search = GridSearchCV(estimator=gb_classifier, param_grid=gb_param_grid, scoring='accuracy', cv=5)
# # gb_grid_search.fit(X_train, y_train)

# # # Get the best hyperparameters and the corresponding estimators for each classifier
# best_params_catboost = catboost_grid_search.best_params_
# best_estimator_catboost = catboost_grid_search.best_estimator_
# # best_params_ada = ada_grid_search.best_params_
# # best_estimator_ada = ada_grid_search.best_estimator_
# # best_params_xgb = xgb_grid_search.best_params_
# # best_estimator_xgb = xgb_grid_search.best_estimator_
# # best_params_gb = gb_grid_search.best_params_
# # best_estimator_gb = gb_grid_search.best_estimator_

# # # Print the best hyperparameters for each classifier
# print("Best Hyperparameters for CatBoost:", best_params_catboost)
# # print("Best Hyperparameters for AdaBoost:", best_params_ada)
# # print("Best Hyperparameters for XGBoost:", best_params_xgb)
# # print("Best Hyperparameters for Gradient Boosting:", best_params_gb)


### Modeling

In [18]:
def Classifiers(X_train, y_train, X_test, threshold=0.05):
    unique_classes = np.unique(y_train)
    class_weights = compute_class_weight(class_weight='balanced', classes=unique_classes, y=y_train)

    classifier = CatBoostClassifier(iterations=110, depth=8, learning_rate=0.01, class_weights=class_weights)

    # Fit the classifier to the training data
    classifier.fit(X_train, y_train)

    # Create a VotingClassifier with soft voting using the specified classifier
    voting_classifier = VotingClassifier(estimators=[
        ('catboost', classifier)
    ], voting='soft')

    # Fit the VotingClassifier to your training data
    voting_classifier.fit(X_train, y_train)

    # Make predictions on the test data using soft voting
    y_test_pred_soft_voting_prob = voting_classifier.predict_proba(X_test)[:, 1]  # Get the probabilities for the positive class
    y_test_pred_soft_voting = (y_test_pred_soft_voting_prob > threshold).astype(int)  # Apply the threshold

    # Create a DataFrame with the soft voting predictions
    data_soft_voting = pd.DataFrame({"ID": range(len(y_test_pred_soft_voting)), "Class": y_test_pred_soft_voting})

    # Save the soft voting predictions to a CSV file
    data_soft_voting.to_csv(f"Result of CatBoostClassifier model.csv", index=False)


In [19]:
Classifiers(X_train, y_train, X_test)

0:	learn: 0.6641428	total: 81.5ms	remaining: 8.88s
1:	learn: 0.6321413	total: 124ms	remaining: 6.7s
2:	learn: 0.6006343	total: 167ms	remaining: 5.94s
3:	learn: 0.5710160	total: 204ms	remaining: 5.41s
4:	learn: 0.5480822	total: 225ms	remaining: 4.71s
5:	learn: 0.5204550	total: 242ms	remaining: 4.19s
6:	learn: 0.4982673	total: 261ms	remaining: 3.83s
7:	learn: 0.4757116	total: 277ms	remaining: 3.54s
8:	learn: 0.4546932	total: 298ms	remaining: 3.34s
9:	learn: 0.4342374	total: 318ms	remaining: 3.18s
10:	learn: 0.4116571	total: 334ms	remaining: 3.01s
11:	learn: 0.3939137	total: 351ms	remaining: 2.87s
12:	learn: 0.3742413	total: 367ms	remaining: 2.74s
13:	learn: 0.3590096	total: 384ms	remaining: 2.63s
14:	learn: 0.3426669	total: 401ms	remaining: 2.54s
15:	learn: 0.3290542	total: 434ms	remaining: 2.55s
16:	learn: 0.3139678	total: 455ms	remaining: 2.49s
17:	learn: 0.2996160	total: 472ms	remaining: 2.41s
18:	learn: 0.2857693	total: 489ms	remaining: 2.34s
19:	learn: 0.2728147	total: 509ms	remaini